In [1]:
import logging
import os
from dotenv import load_dotenv

from xcontest import login, flight_days

load_dotenv()

logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG_LEVEL'] = "0"
os.environ['WDM_LOG'] = "false"

driver = login()

fligtht_day_list= flight_days(driver, 'Metzingen')

#get_launch_day(driver, 'Metzingen','2023-07-07')



['29.09.2024', '22.09.2024', '31.08.2024', '26.08.2024', '03.08.2024', '18.07.2024', '14.07.2024', '29.06.2024', '26.05.2024', '25.05.2024', '11.05.2024', '04.05.2024', '21.04.2024', '31.03.2024', '10.02.2024', '09.09.2023', '11.08.2023', '05.08.2023', '07.07.2023', '18.06.2023', '10.06.2023', '04.06.2023', '03.06.2023', '28.05.2023', '27.05.2023', '18.05.2023', '30.04.2023', '23.04.2023', '07.04.2023', '18.03.2023', '30.10.2022', '09.10.2022', '30.09.2022', '21.08.2022', '02.07.2022', '04.06.2022', '03.06.2022', '18.04.2022', '06.03.2022', '15.01.2022', '24.10.2021', '17.10.2021', '10.10.2021', '02.10.2021', '21.08.2021', '12.08.2021', '25.07.2021', '10.07.2021', '04.07.2021', '27.06.2021', '26.06.2021', '01.06.2021', '30.05.2021', '08.05.2021', '01.05.2021', '17.10.2020', '20.09.2020', '17.09.2020', '19.08.2020', '09.08.2020', '25.07.2020', '18.07.2020', '24.06.2020', '02.06.2020', '01.06.2020', '30.05.2020', '29.05.2020', '21.05.2020', '14.05.2020', '01.01.2020', '31.10.2019', '21.0

In [20]:
import pandas as pd 
from datetime import datetime

df = pd.DataFrame(fligtht_day_list,columns=['isodate'])

df['isodate'] = df.apply(lambda row: datetime.strptime(row['isodate'], '%d.%m.%Y') ,axis=1)
df['year'] = df.apply(lambda row: row['isodate'].year ,axis=1)
df = df[df['year'] >= 2019]

df.groupby(by='year').count().aggregate(['count','mean','std'])


,isodate
count,6.000000
mean,15.333333
std,3.829708
